In [1]:
# general tools
import sys
from glob import glob

# data tools
import time
import h5py
import random
import numpy as np
from random import shuffle

import matplotlib.pyplot as plt
%matplotlib inline

In [2]:
# deep learning tools
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
tf.config.run_functions_eagerly(True)

from keras_unet_collection import models as k_models
from keras_unet_collection import utils as k_utils

2022-09-05 10:37:04.456244: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcudart.so.10.1


In [3]:
sys.path.insert(0, '/glade/u/home/ksha/NCAR/')
sys.path.insert(0, '/glade/u/home/ksha/NCAR/libs/')

from namelist import *
import data_utils as du
import graph_utils as gu
import convnext_keras as ck

from sklearn.metrics import classification_report, auc, roc_curve
from sklearn.metrics import confusion_matrix

import matplotlib.pyplot as plt
%matplotlib inline

In [4]:
def test_metric(VALID_target, Y_pred, thres=0.5):

    tn, fp, fn, tp = confusion_matrix(VALID_target.ravel(), Y_pred.ravel()>thres).ravel()

    return tn, fp, fn, tp


def verif_metric(VALID_target, Y_pred, thres=0.5):

    tn, fp, fn, tp = confusion_matrix(VALID_target.ravel(), Y_pred.ravel()>thres).ravel()

    CSI = tp/(tp+fn+fp)
    CSI_metric = 1 - CSI

    fpr, tpr, thresholds = roc_curve(VALID_target.ravel(), Y_pred.ravel())
    AUC = auc(fpr, tpr)
    AUC_metric = 1 - AUC
    
    freq = (tp+fp)/(tp+fn)
    freq_metric = np.abs(freq-1.0)
    
    print('{} {} {}'.format(CSI, AUC, freq))
    metric = CSI_metric


    return metric

In [5]:
with h5py.File(save_dir_scratch+'VALID_real_lead{}.hdf'.format(21), 'r') as h5io:
    TEST_input = h5io['TEST_input'][...]
    TEST_target = h5io['TEST_target'][...]

TEST_target[np.isnan(TEST_target)] = 0.0
TEST_target[TEST_target!=0] = 1.0

TEST_input = TEST_input[10000:15000, ...]
TEST_target = TEST_target[10000:15000, ...]

In [6]:
MODEL_CONFIGS = {

    "small": {
        "depths": [3, 3, 27, 3],
        "projection_dims": [96, 192, 384, 768],
}}

model = ck.ConvNeXt(
        depths=MODEL_CONFIGS["small"]["depths"],
        projection_dims=MODEL_CONFIGS["small"]["projection_dims"],
        drop_path_rate=0.0,
        layer_scale_init_value=0.1,
        model_name='test',
        input_shape=(128, 128, 19),
        pooling='max',
        classes=1,)#0.5 #0.1 #1e-6

2022-09-05 10:39:34.962773: I tensorflow/compiler/jit/xla_cpu_device.cc:41] Not creating XLA devices, tf_xla_enable_xla_devices not set
2022-09-05 10:39:35.233788: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcuda.so.1
2022-09-05 10:39:35.319259: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1720] Found device 0 with properties: 
pciBusID: 0000:62:00.0 name: Tesla V100-SXM2-32GB computeCapability: 7.0
coreClock: 1.53GHz coreCount: 80 deviceMemorySize: 31.75GiB deviceMemoryBandwidth: 836.37GiB/s
2022-09-05 10:39:35.319323: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcudart.so.10.1
2022-09-05 10:39:35.488955: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcublas.so.10
2022-09-05 10:39:35.490452: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcublasLt.so.10
2

In [7]:
batch_dir = '/glade/scratch/ksha/DATA/NCAR_batch/'
temp_dir = '/glade/work/ksha/NCAR/Keras_models/'

W_old = k_utils.dummy_loader(temp_dir+'CONVNEXT_Base_pp19_tune3')
model.set_weights(W_old)

model.compile(loss=keras.losses.mean_absolute_error, optimizer=keras.optimizers.SGD(lr=0))

In [8]:
#

In [9]:
Y_pred = model.predict([TEST_input,])

Y_pred[Y_pred<0.1] = 0
Y_pred[Y_pred>1] = 1

/glade/work/ksha/anaconda3/lib/python3.9/site-packages/tensorflow/python/data/ops/dataset_ops.py:3503: UserWarning: Even though the tf.config.experimental_run_functions_eagerly option is set, this option does not apply to tf.data functions. tf.data functions are still traced and executed as graphs.
  warnings.warn(
2022-09-05 10:48:48.847461: I tensorflow/compiler/mlir/mlir_graph_optimization_pass.cc:116] None of the MLIR optimization passes are enabled (registered 2)
2022-09-05 10:48:48.868974: I tensorflow/core/platform/profile_utils/cpu_utils.cc:112] CPU Frequency: 2600000000 Hz
2022-09-05 10:48:49.694826: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcudnn.so.7
2022-09-05 10:48:55.193249: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcublas.so.10


In [10]:
verif_metric(TEST_target, Y_pred, thres=0.5)

0.07623318385650224 0.7989000128118308 6.2727272727272725


0.9237668161434978

In [11]:
test_metric(TEST_target, Y_pred, thres=0.5)
#tn, fp, fn, tp

(4777, 190, 16, 17)

In [ ]:
L_valid = len(TEST_input)

rounds = 2
N_var = 19
AUC_drop = np.empty((N_var, 4, 2))

for r in range(rounds):
    for n in range(N_var):
        start_time = time.time()
        print('shuffling var {}'.format(n))

        ind_ = du.shuffle_ind(L_valid)
        TEST_input_shuffle = np.copy(TEST_input)
        TEST_input_shuffle[..., n] = TEST_input_shuffle[ind_, ..., n]

        Y_pred = model.predict([TEST_input_shuffle,])
        
        Y_pred[Y_pred<0.1] = 0
        Y_pred[Y_pred>1] = 1
        
        AUC_drop[n, :, r] = test_metric(TEST_target, Y_pred, thres=0.5)
        print("--- %s seconds ---" % (time.time() - start_time))

np.save(temp_dir+model_name+'_FI.npy', AUC_drop)

shuffling var 0


/glade/work/ksha/anaconda3/lib/python3.9/site-packages/tensorflow/python/data/ops/dataset_ops.py:3503: UserWarning: Even though the tf.config.experimental_run_functions_eagerly option is set, this option does not apply to tf.data functions. tf.data functions are still traced and executed as graphs.
  warnings.warn(


--- 2870.4319467544556 seconds ---
shuffling var 1


In [ ]:
3000/60/60

In [22]:
AUC_drop[0, :, r]

array([4.68801253e-310, 0.00000000e+000, 2.37151510e-322, 4.68803788e-310])

In [48]:
names = [
    'Maximum/Composite radar reflectivity:dB (instant):lambert:atmosphere:level 0',
    'MSLP (MAPS System Reduction):Pa (instant):lambert:meanSea:level 0',
    'AGL reflectivity (max):lambert:heightAboveGround:level 1000 m',
    'Updraft helicity (max):lambert:heightAboveGroundLayer:levels 5000-2000 m',
    'Updraft helicity (max):lambert:heightAboveGroundLayer:levels 2000-0 m',
    'Updraft helicity (max):lambert:heightAboveGroundLayer:levels 3000-0 m',
    'Vorticity (relative):s**-1 (max):lambert:heightAboveGroundLayer:levels 2000-0 m',
    'Vorticity (relative):s**-1 (max):lambert:heightAboveGroundLayer:levels 1000-0 m',
    'Column integrated graupel mass (max):lambert:atmosphereSingleLayer:level 0 considered as a single layer',
    '2 metre temperature:K (instant):lambert:heightAboveGround:level 2 m',
    '2 metre dewpoint temperature:K (instant):lambert:heightAboveGround:level 2 m',
    '10 metre U wind component:m s**-1 (instant):lambert:heightAboveGround:level 10 m',
    '10 metre V wind component:m s**-1 (instant):lambert:heightAboveGround:level 10 m',
    '10 metre wind speed:m s**-1 (max):lambert:heightAboveGround:level 10 m',
    'Total Precipitation:kg m**-2 (accum):lambert:surface:level 0',
    'Convective available potential energy:J kg**-1 (instant):lambert:surface:level 0',
    'Convective inhibition:J kg**-1 (instant):lambert:surface:level 0',
    'Storm relative helicity:J kg**-1 (instant):lambert:heightAboveGroundLayer:levels 3000-0 m',
    'Storm relative helicity:J kg**-1 (instant):lambert:heightAboveGroundLayer:levels 1000-0 m',
    'Vertical u-component shear:s**-1 (instant):lambert:heightAboveGroundLayer:levels 0-1000 m',
    'Vertical v-component shear:s**-1 (instant):lambert:heightAboveGroundLayer:levels 0-1000 m',
    'Vertical u-component shear:s**-1 (instant):lambert:heightAboveGroundLayer:levels 0-6000 m',
    'Vertical v-component shear:s**-1 (instant):lambert:heightAboveGroundLayer:levels 0-6000 m'
]

name_pick = []
for i in [0, 1, 3, 4, 5, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22]:
    print(names[i])
    name_pick.append(names[i])
    
name_pick.append('Climatology')

Maximum/Composite radar reflectivity:dB (instant):lambert:atmosphere:level 0
MSLP (MAPS System Reduction):Pa (instant):lambert:meanSea:level 0
Updraft helicity (max):lambert:heightAboveGroundLayer:levels 5000-2000 m
Updraft helicity (max):lambert:heightAboveGroundLayer:levels 2000-0 m
Updraft helicity (max):lambert:heightAboveGroundLayer:levels 3000-0 m
2 metre temperature:K (instant):lambert:heightAboveGround:level 2 m
2 metre dewpoint temperature:K (instant):lambert:heightAboveGround:level 2 m
10 metre U wind component:m s**-1 (instant):lambert:heightAboveGround:level 10 m
10 metre V wind component:m s**-1 (instant):lambert:heightAboveGround:level 10 m
10 metre wind speed:m s**-1 (max):lambert:heightAboveGround:level 10 m
Total Precipitation:kg m**-2 (accum):lambert:surface:level 0
Convective available potential energy:J kg**-1 (instant):lambert:surface:level 0
Convective inhibition:J kg**-1 (instant):lambert:surface:level 0
Storm relative helicity:J kg**-1 (instant):lambert:heightAb

In [52]:
np.argsort(np.abs(AUC_drop[:, 0]))

array([10,  4,  2,  3, 14, 19, 15,  9,  6,  8,  7,  0, 13, 16,  1,  5, 17,
       18, 12, 11])

In [57]:
np.abs(AUC_drop[0, 0])

0.038819081730771754

In [49]:
for i in range(20):
    if np.abs(AUC_drop[i]) > 0.04:
        print('{} --- {}'.format(np.abs(AUC_drop[i]), name_pick[i]))

[0.05299142] --- MSLP (MAPS System Reduction):Pa (instant):lambert:meanSea:level 0
[0.05688881] --- 2 metre temperature:K (instant):lambert:heightAboveGround:level 2 m
[0.17339486] --- Convective available potential energy:J kg**-1 (instant):lambert:surface:level 0
[0.06764561] --- Convective inhibition:J kg**-1 (instant):lambert:surface:level 0
[0.04045835] --- Storm relative helicity:J kg**-1 (instant):lambert:heightAboveGroundLayer:levels 3000-0 m
[0.04131814] --- Vertical v-component shear:s**-1 (instant):lambert:heightAboveGroundLayer:levels 0-1000 m
[0.0588257] --- Vertical u-component shear:s**-1 (instant):lambert:heightAboveGroundLayer:levels 0-6000 m
[0.06538276] --- Vertical v-component shear:s**-1 (instant):lambert:heightAboveGroundLayer:levels 0-6000 m


**Final decision**

* Maximum/Composite radar reflectivity:dB (instant):lambert:atmosphere:level 0
* MSLP (MAPS System Reduction):Pa (instant):lambert:meanSea:level 0
* 2 metre temperature:K (instant):lambert:heightAboveGround:level 2 m
* Convective available potential energy:J kg**-1 (instant):lambert:surface:level 0
* Convective inhibition:J kg**-1 (instant):lambert:surface:level 0
* Storm relative helicity:J kg**-1 (instant):lambert:heightAboveGroundLayer:levels 3000-0 m
* Vertical u-component shear:s**-1 (instant):lambert:heightAboveGroundLayer:levels 0-6000 m
* Vertical v-component shear:s**-1 (instant):lambert:heightAboveGroundLayer:levels 0-6000 m

In [44]:
np.array(name_pick)[np.abs(AUC_drop[i])>0.04]

IndexError: boolean index did not match indexed array along dimension 0; dimension is 20 but corresponding boolean dimension is 1

In [45]:
np.array(name_pick)

array(['Maximum/Composite radar reflectivity:dB (instant):lambert:atmosphere:level 0',
       'MSLP (MAPS System Reduction):Pa (instant):lambert:meanSea:level 0',
       'Updraft helicity (max):lambert:heightAboveGroundLayer:levels 5000-2000 m',
       'Updraft helicity (max):lambert:heightAboveGroundLayer:levels 2000-0 m',
       'Updraft helicity (max):lambert:heightAboveGroundLayer:levels 3000-0 m',
       '2 metre temperature:K (instant):lambert:heightAboveGround:level 2 m',
       '2 metre dewpoint temperature:K (instant):lambert:heightAboveGround:level 2 m',
       '10 metre U wind component:m s**-1 (instant):lambert:heightAboveGround:level 10 m',
       '10 metre V wind component:m s**-1 (instant):lambert:heightAboveGround:level 10 m',
       '10 metre wind speed:m s**-1 (max):lambert:heightAboveGround:level 10 m',
       'Total Precipitation:kg m**-2 (accum):lambert:surface:level 0',
       'Convective available potential energy:J kg**-1 (instant):lambert:surface:level 0',
    